In [3]:

import matplotlib.pyplot as plt
import gpflow
import gpflux
import numpy as np
import ruptures as rpt
import torch
import torch.nn as nn
import torch.nn.functional as F

import tensorflow as tf
from gpflux.helpers import construct_basic_kernel, construct_basic_inducing_variables
from gpflux.layers import GPLayer
from gpflux.experiment_support.plotting import plot_layer
from scipy.optimize import linear_sum_assignment
from ruptures.base import BaseCost
import random
import math
from torch.utils.data import Dataset, DataLoader
from scipy.stats import wasserstein_distance

# Signal/Time Series Construction
## Poisson Point Process
We start by defining an intensity function for the Non-Homogeneous Poisson process (NHPP) and then generate a Point process using a thinning algorithm that can be found here (https://www.columbia.edu/~ks20/4703-Sigman/4703-07-Notes-PP-NSPP.pdf)

In [4]:
def lam(x, a):
    """
    Intensity function for the NHPP
    """
    return a + np.sin(x)

In [5]:
def sample_poisson_jumps(a, lambda_star, length=1000, seed=None):
    """
    Thinning algorithm for generating a poisson point process signal.
    """
    if seed is not None:
        np.random.seed(seed)
    T = 0.01 * length
    t = 0.0
    n = 0
    arrival_times = []
    while True:
        u = np.random.rand()
        t = t - (1/lambda_star) * np.log(u)

        if t > T:
            break
        u = np.random.rand()
        lambda_t = lam(t, a)
        if u <= lambda_t / lambda_star:
            n += 1
            arrival_times.append(t)
    ts = np.linspace(0, T, length)
    ys = np.searchsorted(arrival_times, ts)
    return ts, ys, arrival_times, T

## Deep Gaussian Process (DGP)
We continue with defining the correlated noise of the signals. We start by defining a helper function for sampling layer kernels and hyperparameters. We choose to sample from four stationary kernels supported by GPFlow for each layer and vary their hyperparameters by sampling from the range [0.1, 5]. We then create a set of datapoints that are propagated through a number of predefined layers and save the output of the final layer.

In [6]:
KERNELS = [gpflow.kernels.SquaredExponential,
          gpflow.kernels.Matern12,
          gpflow.kernels.Matern32,
          gpflow.kernels.Matern52]

In [7]:
def randomly_sample_kernels(kernels, num_kernels, seed=None):
    if seed is not None:
        np.random.seed(seed)
    sampled_kernels = []
    for i in range(num_kernels):
        kernel = np.random.choice(kernels)()
        lengthscales = np.random.uniform(0.1, 5.0)
        variance = np.random.uniform(0.1, 5.0)
        kernel.lengthscales.assign(lengthscales)
        kernel.variance.assign(variance)
        sampled_kernels.append(kernel)
    return sampled_kernels

In [8]:
def sample_dgp_signal(datapoints, a, b, samples, kernels, num_layers, seed=None):
    if seed is not None:
        np.random.seed(seed)
        tf.random.set_seed(seed)
    X = np.linspace(a, b, datapoints).reshape(-1, 1)
    Z = X.copy()
    M = Z.shape[0]
    gp_layers = []
    D = 1

    ## Construct the each layer with inducing variables
    for i in range(num_layers):
        ind_var = construct_basic_inducing_variables(M, D, D, share_variables=True, z_init=Z.copy())
        kernel = construct_basic_kernel(
            kernels[i],
            output_dim=D,
            share_hyperparams=True)
        gplayer = GPLayer(kernel, ind_var, datapoints, full_cov=True, num_samples=samples, mean_function=gpflow.mean_functions.Zero())
        gp_layers.append(gplayer)
    means, covs, samples = [], [], []
    layer_input = X
    for layer in gp_layers:
        layer_output = layer(layer_input)

        mean = layer_output.mean()
        cov = layer_output.covariance()
        sample = tf.convert_to_tensor(layer_output)  # generates num_samples samples...

        layer_input = sample[0]  # for the next layer

        means.append(mean.numpy().T)  # transpose to go from [1, N] to [N, 1]
        covs.append(cov.numpy())
        samples.append(sample.numpy())

    return np.squeeze(samples[num_layers-1])

Next is the definition of a batching function for extracting sequences to train the transformer model on. The final output is of dimension [batch_size, seq_len, 1]

In [9]:
def sequence_batch(samples, seq_len, batch_size, device=None):
    
    # Sample ids from each sample
    indexes = torch.randint(0, len(samples), (batch_size,))

    x_batch = []
    y_batch = []
    for i in indexes:
        series = samples[i]
        N = series.shape[0]

        i = torch.randint(0, N - seq_len - 1, (1,)).item()
        
        x = series[i: i + seq_len]
        y = series[i + 1: i + seq_len + 1]

        x_batch.append(x)
        y_batch.append(y)

    x = torch.stack(x_batch).unsqueeze(-1)
    y = torch.stack(y_batch).unsqueeze(-1)
    if device is not None:
        x = x.to(device)
        y = y.to(device)

    return x,y 

We can now define a function that will generate our sequences. The idea is to sample a single baseline poisson point process and then sample multiple noise signals. In the end we get a Tensor of shape [num_samples, length]

In [10]:
def sequences(a, lambda_star, length, kernels, num_layers, num_samples,poisson_seed=None, kernel_seed=None, dgp_seed=None):
    xs, baseline, times, T = sample_poisson_jumps(a, lambda_star, length, poisson_seed)
    kernels = randomly_sample_kernels(kernels, num_layers, kernel_seed)
    samples = []
    for i in range(num_samples):
        samples.append(baseline + np.squeeze(sample_dgp_signal(len(xs), 0, T, 1, kernels, num_layers, dgp_seed)))
    split = int(0.8 * len(samples))
    samples = torch.tensor(
        np.stack(samples, axis=0),
        dtype=torch.float32
    )
    print("Data generation completed")
    return samples[:split], samples[split:]

In [12]:
class DatasetBuilder(Dataset):
    def __init__(self, samples, seq_len, stride=None, mode = 'deterministic'):
        self.samples = samples
        self.stride = stride if stride is not None else seq_len
        # seq_len here will give us windows
        self.seq_len = seq_len
        self.mode = mode
        self.windows = []
        if mode == 'deterministic':
            self.compute_valid_windows()

    def compute_valid_windows(self):
        num_samples, length = self.samples.shape

        for sample_idx in range(num_samples):
            start = 0
            while start + self.seq_len <= length:
                self.windows.append((sample_idx, start))
                start += self.stride

    def __len__(self):
        return len(self.windows)

    def __getitem__(self, idx):
        sample_idx, start = self.windows[idx]

        window = self.samples[sample_idx, start : start + self.seq_len]
        # [window_length, 1]
        x = window[:-1].unsqueeze(-1) 
        y = window[1:].unsqueeze(-1)   

        return x, y

# Transformer
We now define a simple decoder-only Transformer model to be trained on the time series or signal data. This implementation is based on (https://huggingface.co/datasets/bird-of-paradise/transformer-from-scratch-tutorial/tree/main). To be able to adapt the architecture for continuous data, we linearly project the sequence points into the embedding dimension and then use a regression head to get the final output.

In [13]:
# Just some global variables
SEQ_LEN = 50
BATCH_SIZE = 16
D_M = 128
STEPS = 100
device = 'cpu'

In [14]:
class MultiHeadAtt(nn.Module):
    def __init__(self, d_m, n_heads, dropout=0.1, bias=False):
        super().__init__()
        self.d_m = d_m
        self.n_heads = n_heads
        self.d_k = d_m // n_heads

        # Full projections d_m*d_m
        self.W_query = nn.Linear(d_m, d_m, bias=bias)
        self.W_key = nn.Linear(d_m, d_m, bias=bias)
        self.W_value = nn.Linear(d_m, d_m, bias=bias)

        self.W_multi = nn.Linear(d_m, d_m, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, sequence_length, d_m = x.shape

        query_matrix = self.W_query(x)
        key_matrix = self.W_key(x)
        value_matrix = self.W_value(x)

        # (B, H, N, D)
        query_heads = query_matrix.view(batch_size, sequence_length, self.n_heads, self.d_k).transpose(1,2)
        key_heads = key_matrix.view(batch_size, sequence_length, self.n_heads, self.d_k).transpose(1,2)
        value_heads = value_matrix.view(batch_size, sequence_length, self.n_heads, self.d_k).transpose(1,2)

        # QK^T
        qkt = torch.matmul(query_heads, key_heads.transpose(-1, -2))/ (self.d_k ** 0.5)

        # mask
        mask = torch.tril(torch.ones(sequence_length, sequence_length, device=x.device)).unsqueeze(0).unsqueeze(0)
        add_mask = (1.0 - mask) * -1e9
        masked_qkt = qkt + add_mask
        #masked_qkt = qkt.masked_fill(mask == 0, float("-inf"))

        # Softmax
        att_weights = F.softmax(masked_qkt, dim = -1)
        
        # Dropout
        att_weights = self.dropout(att_weights)

        # QK^T * V
        attention = torch.matmul(att_weights, value_heads)

        # Concatenate
        output_matrix = attention.transpose(1, 2).contiguous().view(batch_size, sequence_length, d_m)

        output = self.W_multi(output_matrix)

        return output

        

In [15]:
class FeedForward(nn.Module):
    def __init__(self, d_m, d_ff, dropout=0.1):
        super().__init__()
        self.w1 = nn.Linear(d_m, d_ff)
        self.w2 = nn.Linear(d_ff, d_m)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.w1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.w2(x)

        return x

In [16]:
class Block(nn.Module):
    def __init__(self, d_m, n_heads):
        super().__init__()
        self.d_m = d_m
        self.d_ff = 4*d_m
        self.attention = MultiHeadAtt(self.d_m, n_heads)
        self.feedforward = FeedForward(self.d_m, self.d_ff)

        self.layernorm_att = nn.LayerNorm(self.d_m)
        self.layernorm_ffn = nn.LayerNorm(self.d_m)
    def forward(self, x):
        x = x + self.attention(self.layernorm_att(x))
        x = x + self.feedforward(self.layernorm_ffn(x))
        return x
        

In [17]:
class PositionalEncodings(nn.Module):
    def __init__(self, d_m, max_len=2000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_m)

        # pos
        position = torch.arange(0, max_len, dtype = torch.float).unsqueeze(1)

        # i
        indexes = torch.arange(0, d_m,step=2, dtype= torch.float)

        pe[:, 0::2] = torch.sin(position/(torch.tensor(10000) **(indexes/d_m)))
        pe[:, 1::2] = torch.cos(position/(torch.tensor(10000) **(indexes/d_m)))
        #Add batch_size
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [18]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, d_input, d_m, n_heads, n_layers, max_len=2000, dropout=0.1):
        super().__init__()
        self.d_m = d_m
        self.d_input = d_input

        # Project time series to embedding dimension
        self.input_linear = nn.Linear(d_input, d_m)

        # Position encodings
        self.pos = PositionalEncodings(d_m)

        decoders = [Block(d_m, n_heads) for i in range(n_layers)]
        self.blocks = nn.ModuleList(decoders)

        self.dropout = nn.Dropout(dropout)
        self.layernorm = nn.LayerNorm(d_m)
        # Project back to input dimension
        self.output_linear = nn.Linear(d_m, d_input)

    def forward(self, x):

        x = self.input_linear(x) * math.sqrt(self.d_m)

        x = self.pos(x)

        x = self.dropout(x)

        for block in self.blocks:
            x = block(x)
        x = self.layernorm(x) # (B, seq_len, d_m)
        x = self.output_linear(x)  # (B, seq_len, 1)
        return x
            
    def generate(self, x, max_points, device='cpu'):
        self.eval()
        with torch.no_grad():
            x = x.to('cpu')
            predictions = []

            for i in range(max_points):
                # Keep context
                x_input = x[:, -x.size(1):, :]
                # Get predictions (B, seq_len, 1)
                pred = self(x_input)
                # Keep only the last one
                next_val = pred[:, -1:, :]
                x = torch.cat([x, next_val], dim=1)
                predictions.append(next_val.cpu())

        return torch.cat(predictions, dim=1)
        

We can now create the training loop for the models

In [81]:
def train(model, train_loader, val_loader,
          learning_rate=0.001, epochs=20):

    model = model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
    mse = nn.MSELoss()

    train_losses = []
    val_losses = []

    for epoch in range(epochs):

        # ---------- Training ----------
        model.train()
        train_loss = 0.0

        for x_train, y_train in train_loader:
            x_train = x_train.to(device)
            y_train = y_train.to(device)

            opt.zero_grad()
            output = model(x_train)
            loss = mse(output, y_train)
            loss.backward()
            opt.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # ---------- Validation ----------
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val = x_val.to(device)
                y_val = y_val.to(device)

                output = model(x_val)
                loss = mse(output, y_val)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"Train: {train_loss:.4f} | "
            f"Val: {val_loss:.4f}"
        )

    return train_losses, val_losses, model

In [19]:
def generate_sine_wave_sequences(
    n_samples=1000,
    length=300,
    n_waves=3,
    split_ratio=0.8,
    seed=None,
):
    """
    Generate sine-wave time series and split into train / validation sets.

    Returns:
        train_samples: torch.FloatTensor (N_train, length)
        val_samples:   torch.FloatTensor (N_val, length)
    """

    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)

    samples = np.zeros((n_samples, length), dtype=np.float32)

    for i in range(n_samples):
        freq = np.random.uniform(0.5, 3.0, n_waves)
        phase = np.random.uniform(0, 2 * np.pi, n_waves)
        amp = np.random.uniform(0.5, 1.5, n_waves)

        t = np.linspace(0, 4 * np.pi, length)
        signal = np.zeros(length, dtype=np.float32)

        for f, p, a in zip(freq, phase, amp):
            signal += a * np.sin(f * t + p)

        signal /= n_waves
        samples[i] = signal

    split = int(split_ratio * n_samples)

    train_samples = torch.FloatTensor(samples[:split])
    val_samples   = torch.FloatTensor(samples[split:])

    return train_samples, val_samples

In [83]:
train_samples, val_samples = generate_sine_wave_sequences()

# Build datasets
train_dataset = DatasetBuilder(
    samples=train_samples,
    seq_len=300,
    stride=300,
    mode="deterministic"
)

val_dataset = DatasetBuilder(
    samples=val_samples,
    seq_len=300,
    stride=300,
    mode="deterministic"
)
print(len(val_dataset))
print(len(train_dataset))

# Build dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=True
)

200
800


In [85]:
model = TimeSeriesTransformer(1, D_M, 4, 3)
train_los, val_los, model = train(model, train_loader, val_loader, epochs=50)
torch.save(model.state_dict(), 'transformer_model.pth')
model.eval()

# pick one validation sample
idx = torch.randint(0, val_samples.shape[0], (1,)).item()
gt_sequence = val_samples[idx].unsqueeze(0).unsqueeze(-1).to(device)  
generated = model.generate(gt_sequence[:, :150,:], 150)
#signal_plot = gt_sequence[:,:50,:].squeeze(0).squeeze(-1).cpu().numpy()
ground_truth = gt_sequence.squeeze(0).squeeze(-1).cpu().numpy()
generated_plot = generated.squeeze(0).squeeze(-1).cpu().numpy()
plt.figure(figsize=(12,4))

plt.plot(ground_truth, label="Ground Truth", linewidth=2)
#plt.plot(generated_plot, label="Generated", linestyle="--")
plt.plot(
    range(150, 150 + len(generated_plot)),
    generated_plot,
    "--",
    label="Generated (future)"
)
plt.axvline(150, color="k", linestyle=":", label="Prediction Start")

plt.legend()
plt.title("Synthetic vs Generated Time Series")
plt.xlabel("Time")
plt.ylabel("Value")
plt.show()

Epoch 1/50 | Train: 0.0731 | Val: 0.0038
Epoch 2/50 | Train: 0.0085 | Val: 0.0014
Epoch 3/50 | Train: 0.0031 | Val: 0.0014
Epoch 4/50 | Train: 0.0025 | Val: 0.0010
Epoch 5/50 | Train: 0.0023 | Val: 0.0010
Epoch 6/50 | Train: 0.0022 | Val: 0.0013
Epoch 7/50 | Train: 0.0020 | Val: 0.0011
Epoch 8/50 | Train: 0.0020 | Val: 0.0011
Epoch 9/50 | Train: 0.0019 | Val: 0.0011
Epoch 10/50 | Train: 0.0019 | Val: 0.0011
Epoch 11/50 | Train: 0.0018 | Val: 0.0011
Epoch 12/50 | Train: 0.0018 | Val: 0.0013
Epoch 13/50 | Train: 0.0017 | Val: 0.0012
Epoch 14/50 | Train: 0.0017 | Val: 0.0014
Epoch 15/50 | Train: 0.0017 | Val: 0.0015
Epoch 16/50 | Train: 0.0017 | Val: 0.0013
Epoch 17/50 | Train: 0.0015 | Val: 0.0017
Epoch 18/50 | Train: 0.0015 | Val: 0.0015
Epoch 19/50 | Train: 0.0015 | Val: 0.0015
Epoch 20/50 | Train: 0.0015 | Val: 0.0015
Epoch 21/50 | Train: 0.0014 | Val: 0.0013
Epoch 22/50 | Train: 0.0014 | Val: 0.0017
Epoch 23/50 | Train: 0.0014 | Val: 0.0014
Epoch 24/50 | Train: 0.0013 | Val: 0.0017
E

KeyboardInterrupt: 

In [20]:
def sequence_batch(samples, seq_len, batch_size, device=None):
    #processing = []
    #for sample in samples:
        #processing.append(torch.tensor(sample, dtype=torch.float32))
    
    # Sample ids from each sample
    indexes = torch.randint(0, len(samples), (batch_size,))

    x_batch = []
    y_batch = []
    for i in indexes:
        series = samples[i]
        N = series.shape[0]

        i = torch.randint(0, N - seq_len - 1, (1,)).item()
        
        x = series[i: i + seq_len]
        y = series[i + 1: i + seq_len + 1]

        x_batch.append(x)
        y_batch.append(y)

    x = torch.stack(x_batch).unsqueeze(-1)
    y = torch.stack(y_batch).unsqueeze(-1)
    if device is not None:
        x = x.to(device)
        y = y.to(device)

    return x,y 

In [21]:
def train(model, train_samples, val_samples, learning_rate=0.001, steps=5000, seed=None):
    model = model.to(device)

    opt = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    sc = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=steps)
    mse = nn.MSELoss()

    train_losses = []
    val_losses = []

    val_interval = 100  # Validate every 100 steps

    for step in range(steps):
        # --- Training ---
        model.train()
        x_train, y_train = sequence_batch(train_samples, 512, BATCH_SIZE, device=device)
        opt.zero_grad()
        output = model(x_train)
        loss = mse(output, y_train)
        loss.backward()
        opt.step()
        sc.step()
        train_losses.append(loss.item())

        # --- Periodic Validation ---
        if (step + 1) % val_interval == 0 or step == 0:
            model.eval()
            with torch.no_grad():
                x_val, y_val = sequence_batch(val_samples, SEQ_LEN, BATCH_SIZE, device=device)
                val_loss = mse(model(x_val), y_val).item()
            val_losses.append((step + 1, val_loss))
            print(f'Step {step+1}/{steps} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss:.4f}')

    return train_losses, val_losses, model

In [23]:
def generate_synthetic_data(n_samples=1000, length=1024):
    t = np.linspace(0, 5, length)
    data = []
    for _ in range(n_samples):
        phase = np.random.rand() * 2*np.pi
        freq = np.random.rand() * 0.5 + 0.1
        signal = np.sin(freq * t + phase) + 0.1*np.random.randn(length)
        data.append(signal)
    return data

samples = generate_synthetic_data(n_samples=100, length=1024)
processing = []
for sample in samples:
    processing.append(torch.tensor(sample, dtype=torch.float32))
n_train = int(0.8 * len(processing)) 
train_samples = processing[:n_train]
val_samples = processing[n_train:]
model = TimeSeriesTransformer(1, D_M, 4, 3)
train_los, val_los, model = train(model, train_samples, val_samples)
model.eval()
signal = random.choice(val_samples)
signal = signal.unsqueeze(0).unsqueeze(-1)
generated = model.generate(signal[:, :50,:], 950)
signal_plot = signal.squeeze(0).squeeze(-1).cpu().numpy()
generated_plot = generated.squeeze(0).squeeze(-1).cpu().numpy()
plt.figure(figsize=(12,4))

plt.plot(signal_plot, label="Ground Truth", linewidth=2)
plt.plot(
    range(50, 50 + len(generated_plot)),
    generated_plot,
    "--",
    label="Generated (future)"
)
plt.axvline(50, color="k", linestyle=":", label="Prediction Start")

plt.legend()
plt.title("Synthetic vs Generated Time Series")
plt.xlabel("Time")
plt.ylabel("Value")
plt.show()

Step 1/5000 | Train Loss: 0.6891 | Val Loss: 0.1404
Step 100/5000 | Train Loss: 0.0220 | Val Loss: 0.0198
Step 200/5000 | Train Loss: 0.0173 | Val Loss: 0.0152
Step 300/5000 | Train Loss: 0.0161 | Val Loss: 0.0143
Step 400/5000 | Train Loss: 0.0156 | Val Loss: 0.0145
Step 500/5000 | Train Loss: 0.0147 | Val Loss: 0.0137
Step 600/5000 | Train Loss: 0.0170 | Val Loss: 0.0141
Step 700/5000 | Train Loss: 0.0139 | Val Loss: 0.0135
Step 800/5000 | Train Loss: 0.0135 | Val Loss: 0.0123
Step 900/5000 | Train Loss: 0.0126 | Val Loss: 0.0143
Step 1000/5000 | Train Loss: 0.0139 | Val Loss: 0.0184
Step 1100/5000 | Train Loss: 0.0121 | Val Loss: 0.0107
Step 1200/5000 | Train Loss: 0.0116 | Val Loss: 0.0121
Step 1300/5000 | Train Loss: 0.0115 | Val Loss: 0.0117
Step 1400/5000 | Train Loss: 0.0115 | Val Loss: 0.0117
Step 1500/5000 | Train Loss: 0.0123 | Val Loss: 0.0123


KeyboardInterrupt: 

## Estimation

We can extract meaningful statistics from each signal using a changepoint estimator.


In [64]:
def estimate_cp(signal, pen):
    return rpt.Pelt(model='l2').fit(signal).predict(pen=pen)

In [73]:
def extract_signal_statistics(ys):
    # First, estimate the changepoints
    cps = estimate_cp(ys, 5)
    cps = np.asarray(cps, dtype=int)
    T = len(ys)
    
    # Get the boundaries of each changepoint
    boundaries = np.concatenate(([0], cps))
    # 1) We calculate the length of the regime change 
    inter_arrivals = np.diff(boundaries)

    # 2) We calculate the rate of changepoints and their number
    cp_rate = len(cps)/T
    num_cps = len(cps)

    means = []
    variances = []
    lengths = inter_arrivals
    segments = [(boundaries[i],boundaries[i+1]) for i in range(len(boundaries)-1)]
    for (start, end) in segments:
        segment = ys[start:end]
        means.append((segment.mean() - ys.mean()) / ys.std())
        variances.append(segment.var())
    # 3) Local statistics, means, variances and lengths
    means = np.asarray(means)
    variances = np.asarray(variances)

    # 4) Jumps in means
    jumps = np.diff(means)

    # 5) correlation of lengths, everything but the last vs everything but the first
    length_corr = (
        np.corrcoef(lengths[:-1], lengths[1:])[0, 1]
        if len(lengths) > 2 else np.nan
    )

    jump_length_corr = (
        np.corrcoef(np.abs(jumps), lengths[1:])[0, 1]
        if len(jumps) > 1 else np.nan
    )

    statistics =  {
        "cps" : cps,
        "num_cps" : num_cps,
        "cp_rate": cp_rate,
        "inter_arrivals": inter_arrivals,
        "lengths": lengths,
        "means": means,
        "variances": variances,
        "jumps": jumps,
        "length_corr": length_corr,
        "jump_length_corr": jump_length_corr,
        }
    return statistics
    